## 构建神经网络

In [1]:
import os
import torch
from keras.src.utils.module_utils import torchvision
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

### 获取训练设备


In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using {device} device")

Using cpu device


### 定义类

我们通过继承 nn.Module 来定义我们的神经网络，并在 __init__ 中初始化神经网络层。每个 nn.Module 子类都在 forward 方法中实现对输入数据的操作。

In [12]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [13]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [14]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([5])


### 其中，从构建模型层开始

首先模拟生成图片输入

In [15]:
input_image = torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])


### nn.Flatten

In [16]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


### nn.Linear


In [17]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


### nn.ReLU

In [18]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[-0.2151, -0.4946,  0.2322, -0.3633, -0.1978, -0.1958,  0.3194, -0.1729,
          0.2270,  0.1978, -0.1490,  0.1652,  0.0130,  0.2478,  0.6004,  0.5054,
          0.1026, -0.4824, -0.2080, -0.4235],
        [-0.2996, -0.2095,  0.3009, -0.0009, -0.2728, -0.2079,  0.1153, -0.0926,
          0.3164,  0.1010, -0.1627,  0.1365, -0.3530,  0.5112, -0.0093, -0.0232,
          0.2357, -0.6794,  0.0710, -0.3191],
        [-0.2474, -0.2013,  0.1729, -0.0458,  0.0051, -0.2627,  0.0679,  0.0391,
          0.2579,  0.2285, -0.6275,  0.2015, -0.1899,  0.6176,  0.3250,  0.4509,
          0.0440, -0.3656, -0.0850, -0.3074]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.0000, 0.0000, 0.2322, 0.0000, 0.0000, 0.0000, 0.3194, 0.0000, 0.2270,
         0.1978, 0.0000, 0.1652, 0.0130, 0.2478, 0.6004, 0.5054, 0.1026, 0.0000,
         0.0000, 0.0000],
        [0.0000, 0.0000, 0.3009, 0.0000, 0.0000, 0.0000, 0.1153, 0.0000, 0.3164,
         0.1010, 0.0000, 0.1365, 0.0000, 0.5112, 0.00

### nn.Sequential
模块的有序容器，数据通过组合的顺序通过所有模块，可用于组合神经网络

In [19]:
seq_models = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20,10)
)
input_image = torch.rand(3,28,28)
logits = seq_models(input_image)


### nn.Softmax


In [20]:
softmax = nn.Softmax(dim = 1)
pred_probab = softmax(logits)

神经网络内部的许多层都是参数化的，即具有在训练期间优化的相关权重和偏置。继承 nn.Module 会自动跟踪模型对象内定义的所有字段，并使所有参数都可以使用模型的 parameters() 或 named_parameters() 方法访问。

在此示例中，我们遍历每个参数，并打印其大小和值的预览。

In [23]:
print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[-0.0070, -0.0195, -0.0182,  ...,  0.0117, -0.0094,  0.0212],
        [ 0.0198,  0.0231, -0.0075,  ...,  0.0279,  0.0352,  0.0136]],
       grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([0.0011, 0.0137], grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[-0.0111,  0.0280,  0.0270,  ...,  0.0202, -0.0033, -0.0283],
        [ 0.0247, -0.0397,  0.0411,  ..., -0.0024,  0.0385,  0.0290]],
       grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.bias | Si